In [14]:
import seaborn
import pandas as pd
import dataframe_image
import datetime
import matplotlib as plt
from datetime import timedelta

In [3]:
def openFile(path):
    data = pd.read_csv(path)
    return data

In [78]:
def cleanUp (data, process, completedKeyword, dropColumns=['Resource', 'Loaded','LastUpdated', 'Exception', 'Deferred','Completed','KeyValue','Priority','Attempt','Locked','ExceptionReason','isLocked']):
    data['isLocked'] = data['Locked'].apply(lambda x: 1 if pd.notnull(x) else 0)
    data['Status'] = data['Status'].apply(lambda x: x if pd.notnull(x) else "Pending")
    data = data.apply(lambda row: setStatus(row, completedKeyword), axis=1)
    data = data.drop(columns=dropColumns)
    data["Process"] = process
    return data

In [79]:
def toString(data):
    print("Complete: " + getStatusCount(data,"Complete"))
    print("Business Exception: " + getStatusCount(data,"Business Exception"))
    print("Locked: " + getStatusCount(data,"Locked"))
    print("Pending " + getStatusCount(data,"Pending"))
    print("Exception " + getStatusCount(data,"Exception"))

In [80]:
def setStatus(row, completedKeyword):
    if pd.notnull(row["Tags"]) and "Business Exception" in row["Tags"]:
        row["Status"] = "Business Exception"
    elif row["isLocked"] == 1:
        row["Status"] = "Locked"
    elif row["Status"] == completedKeyword:
        row["Status"] = "Complete"
    elif row["Status"] == "Pending":
        pass
    else:
        row["Status"] = "Exception"
    return row

In [81]:
def getStatusCount(data,status):
    count, _ = data[data["Status"] == status].shape
    return str(count)

In [82]:
def getWorktime(row):
    minutes = int(row["Worktime"]/60)
    seconds = (row["Worktime"]%60)
    timespan = timedelta(minutes=minutes,seconds=seconds)
    row["Worktime"] = timespan
    return row

In [83]:
directory = "resources/reports/08-26/"
filename = "icd.csv"
path = directory + filename
data = openFile(path)
cleandf = cleanUp(data,"ICD","Complete")
cleandf[cleandf["Status"]=="Business Exception"]
toString(cleandf)

Complete: 2802
Business Exception: 117
Locked: 10
Pending 54
Exception 814
